In [2]:
import pandas as pd

In [3]:
ret_sample_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'

df = pd.read_csv(ret_sample_path)

print(df.head)

C:\Users\carte\AppData\Local\Temp\ipykernel_10816\4076849072.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ret_sample_path)


<bound method NDFrame.head of                       id      date   ret_eom     gvkey  iid excntry  \
0        comp_001081_01C  20050228  20050228    1081.0  01C     CAN   
1        comp_001096_01C  20050228  20050228    1096.0  01C     CAN   
2         comp_001117_02  20050228  20050228    1117.0   02     USA   
3        comp_001166_01W  20050228  20050228    1166.0  01W     NLD   
4        comp_001186_01C  20050228  20050228    1186.0  01C     CAN   
...                  ...       ...       ...       ...  ...     ...   
6401409  comp_367287_01W  20250630  20250630  367287.0  01W     HKG   
6401410  comp_367313_01W  20250630  20250630  367313.0  01W     DEU   
6401411  comp_367333_01W  20250630  20250630  367333.0  01W     ITA   
6401412  comp_367334_01W  20250630  20250630  367334.0  01W     ITA   
6401413  comp_367496_01W  20250630  20250630  367496.0  01W     ESP   

         stock_ret  year  month  char_date  ...  betadown_252d  \
0        -0.143457  2005      2   20050131  ...    

In [ ]:
import pandas as pd
import os

def process_csv_in_chunks(input_csv_path, output_dir, chunk_size=1000000):
    """
    Reads and processes a large CSV in chunks to avoid memory errors. It
    uses the 'gvkey' column to save separate CSV files for each unique ID.

    Args:
        input_csv_path (str): The path to the input CSV file.
        output_dir (str): The directory where the output CSVs will be saved.
        chunk_size (int): The number of rows to process in each chunk.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # This set will keep track of which files we have already written the header for
    created_files = set()
    
    try:
        print(f"Reading data from {input_csv_path} in chunks of {chunk_size} rows...")
        
        # Use the 'chunksize' parameter to create an iterator
        chunk_iterator = pd.read_csv(input_csv_path, low_memory=False, chunksize=chunk_size)
        
        # Loop through each chunk (a smaller DataFrame) from the file
        for i, chunk in enumerate(chunk_iterator):
            print(f"--- Processing Chunk {i+1} ---")

            # --- ID Transformation using 'gvkey' column ---
            print("Cleaning 'gvkey' column and preparing for grouping...")
            # Drop rows where gvkey is missing
            chunk.dropna(subset=['gvkey'], inplace=True)
            # Convert gvkey to integer type to remove decimals (e.g., 1081.0 -> 1081)
            chunk['gvkey'] = chunk['gvkey'].astype(int)
            
            # --- Append rows to the correct files ---
            print("Grouping by 'gvkey' and saving to files...")
            # Group the current chunk by the integer 'gvkey'
            grouped_chunk = chunk.groupby('gvkey')
            
            # Loop through each unique ID found in this chunk
            for current_id, group_df in grouped_chunk:
                file_name = f"{current_id}.csv"
                output_path = os.path.join(output_dir, file_name)
                
                # If we haven't created this file yet, write with a header.
                # Otherwise, append without the header.
                if output_path not in created_files:
                    group_df.to_csv(output_path, index=False, mode='w', header=True)
                    created_files.add(output_path) # Mark this file as created
                else:
                    group_df.to_csv(output_path, index=False, mode='a', header=False)
            
            print(f"Chunk {i+1} processed successfully.")
            # Add a running counter after each chunk is processed
            print(f"Unique IDs found so far: {len(created_files)}")

        print("\nProcessing complete. All chunks have been processed and files saved.")
        # Add a final count at the end of the script
        print(f"Total unique IDs found and files created: {len(created_files)}")

    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found. Please check the path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- How to use this script ---
if __name__ == "__main__":
    print("Starting the memory-efficient script to split data by company ID.")
    
    # 1. Set the path to your input CSV file.
    input_file_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'
    
    # 2. Set the path for the output folder.
    output_folder_name = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\csv_by_id'
    
    # 3. Run the main function.
    process_csv_in_chunks(input_file_path, output_folder_name)



Starting the memory-efficient script to split data by company ID.
Created output directory: C:\_Files\Personal\Projects\FIAM\FIAM2025\data\csv_by_id
Reading data from C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv in chunks of 500000 rows...
--- Processing Chunk 1 ---
Extracting and cleaning IDs...
Grouping by ID and saving to files...
Chunk 1 processed successfully.
Unique IDs found so far: 20599
--- Processing Chunk 2 ---
Extracting and cleaning IDs...
Grouping by ID and saving to files...
Chunk 2 processed successfully.
Unique IDs found so far: 23736
--- Processing Chunk 3 ---
Extracting and cleaning IDs...
Grouping by ID and saving to files...


KeyboardInterrupt: 